In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import spacy
from datasets import load_dataset
from transformers import MarianTokenizer
from tqdm import tqdm
from collections import defaultdict

we need to make sure the character-based span matching will _actually_ match correctly

In [ ]:
def get_spans(text, tokens):
    spans = []
    i = 0
    for token in tokens:
        while i < len(text) and text[i].isspace(): i += 1
        if i < len(text):
            s = i
            i = text.find(token, i) + len(token)
            spans.append((s, i))
    
    assert len(spans) == len(tokens), f'number of spans does not match number of tokens for: {text}'
    return spans

def get_alignments(text, spacy_tokens, marian_tokens):
    spacy_spans = get_spans(text, spacy_tokens)
    marian_spans = get_spans(text, marian_tokens)
    alignment = {} # map marian_tokens[i] to spacy_tokens[j]
    best_overlap = defaultdict(int) # track max overlap
    
    # just bruteforce check (who needs DP?)
    for i, marian_span in enumerate(marian_spans):
        for j, spacy_span in enumerate(spacy_spans):
            overlap = max(0, min(spacy_span[1], marian_span[1]) - max(spacy_span[0], marian_span[0]))
            if overlap > 0 and overlap > best_overlap[i]:
                alignment[i] = j
                best_overlap[i] = overlap
                
    for i, tok in enumerate(marian_tokens):
        if tok == '': alignment[i] = alignment[i+1] # word break
        if tok == '<unk>': alignment[i] = 1 + alignment[i-1] # unk
    
    assert len(alignment) == len(marian_tokens), f'did not find a complete alignment for: {text}'
    return alignment

marian = MarianTokenizer.from_pretrained('Helsinki-NLP/opus-mt-en-de')
# nlp = spacy.load('en_core_web_sm')
nlp = spacy.load('de_core_news_sm')
wmt = load_dataset('wmt14', 'de-en', split='validation')

def get_spacy_tokens(text):
    return [token.text for token in nlp(text)]

def get_marian_tokens(text):
    return [marian.decode(id) for id in marian(text)['input_ids'][:-1]]

for i, example in enumerate(tqdm(wmt['translation'])):
    # text = example['en']
    text = example['de']
    spacy_tokens = get_spacy_tokens(text)
    marian_tokens = get_marian_tokens(text)
    alignments = get_alignments(text, spacy_tokens, marian_tokens)

In [ ]:
class Parser:

    def __init__(self):
        self.en_nlp = spacy.load('en_core_web_sm')
        self.de_nlp = spacy.load('de_core_news_sm')
        self.marian = MarianTokenizer.from_pretrained('Helsinki-NLP/opus-mt-en-de')
    
    def get_spans(self, text, tokens):
        spans = []
        i = 0
        for token in tokens:
            while i < len(text) and text[i].isspace(): i += 1
            if i < len(text):
                s = i
                i = text.find(token, i) + len(token)
                spans.append((s, i))
        
        assert len(spans) == len(tokens), f'number of spans does not match number of tokens for: {text}'
        return spans

    def get_alignment(self, text, spacy_tokens, marian_tokens, spacy_spans=None, marian_spans=None):
        if spacy_spans is None: spacy_spans = self.get_spans(spacy_tokens) 
        if marian_tokens is None: marian_spans = self.get_spans(marian_tokens) 
        
        alignment = {} # map marian_tokens[i] to spacy_tokens[j]
        best_overlap = defaultdict(int) # track max overlap
        
        # just bruteforce check (who needs DP?)
        for i, marian_span in enumerate(marian_spans):
            for j, spacy_span in enumerate(spacy_spans):
                overlap = max(0, min(spacy_span[1], marian_span[1]) - max(spacy_span[0], marian_span[0]))
                if overlap > 0 and overlap > best_overlap[i]:
                    alignment[i] = j
                    best_overlap[i] = overlap
                    
        for i, tok in enumerate(marian_tokens):
            if tok == '': alignment[i] = alignment[i+1] # word break
            if tok == '<unk>': alignment[i] = 1 + alignment[i-1] # unk
        
        assert len(alignment) == len(marian_tokens), f'did not find a complete alignment for: {text}'
        return alignment

    def get_spacy_tokens(self, text, lang):
        return [token.text for token in (self.en_nlp if lang == 'en' else self.de_nlp)(text)]

    def get_marian_tokens(self, text):
        return [self.marian.decode(id) for id in self.marian(text)['input_ids'][:-1]]
    
    def align(self, text, lang):
        spacy_tokens = self.get_spacy_tokens(text, lang)
        spacy_spans = self.get_spans(text, spacy_tokens)
        normalized_marian_tokens = self.get_marian_tokens(text)
        marian_spans = self.get_spans(text, normalized_marian_tokens)

        raw_marian_tokens = self.marian.tokenize(text)
        doc = (self.en_nlp if lang == 'en' else self.de_nlp)(text)
        alignment = self.get_alignment(text, spacy_tokens, normalized_marian_tokens, spacy_spans, marian_spans)
    
        # reverse map spacy to marian tokens
        reverse = defaultdict(list)
        for k, v in alignment.items(): reverse[v].append(k)

        # get original root idx
        root_i = next(i for i, tok in enumerate(doc) if tok.head == tok)

        seq = []
        for i, text in enumerate(raw_marian_tokens):
            spacy_tok = doc[alignment[i]]
            seq.append({'text': text, 'pos': spacy_tok.pos_, 'i': i, 'is_head': alignment[i] == root_i})
        
        for i, _ in enumerate(seq):
            spacy_children = doc[alignment[i]].children
            seq[i]['children'] = []
            for child in spacy_children:
                seq[i]['children'].extend(seq[i] for i in reverse[child.i])
            
            ### heuristic for lineage 
            # spacy_parent = doc[alignment[i]].head.i
            # inferred_parent = reverse[spacy_parent][0]
            seq[i]['par'] = reverse[doc[alignment[i]].head.i][0]
            
        return seq

In [ ]:
from mt import WMTForEvolver

parser = Parser()
wmt = WMTForEvolver(split='validation')

traj = wmt._get_short_input_traj(parser.align(text, 'en'))

for thing in traj:
    print(thing)

In [ ]:
text = 'The new restrictions disproportionately affect young people, minorities and people with low incomes.'

seq = parser.align(text, 'en')
for thing in seq:
    print(thing['text'], '<-', seq[thing['par']]['text'])

In [ ]:
en_nlp = spacy.load('en_core_web_sm')

for token in en_nlp(text):
    # print({'text': token.text, 'pos': token.pos_, 'i': token.i, 'is_head': token == token.head})
    print(token.text, '<-', token.head.text)

test loader ts

In [ ]:
from mt import WMTForEvolver

dataset = WMTForEvolver(split='validation')

In [ ]:
item = dataset[0]

test that everything we load actually applies to create the correct output

In [ ]:
import torch

def apply_edits(input_ids, edit_ids):
    op_ids, tok_ids, idx_ids = edit_ids
    res = torch.zeros(op_ids.shape[1], dtype=torch.long)
    ins_mask = op_ids.eq(0) | op_ids.eq(2)
    res[ins_mask] = tok_ids[ins_mask]
    cpy_mask = op_ids.eq(1)
    permuted_inputs = input_ids[torch.arange(1).view(-1, 1), idx_ids]
    res[cpy_mask] = permuted_inputs[cpy_mask]
    return res

debug index error in parser

In [ ]:
from data import MarianTokenizer

tokenizer = MarianTokenizer.from_pretrained('Helsinki-NLP/opus-mt-en-de')

In [ ]:
from tqdm import tqdm

bad_apples = 0

for i in tqdm(range(len(dataset))):
    try: _ = dataset[i]
    except IndexError:
        print(dataset.dataset[i]['translation']['en'])
        bad_apples += 1

In [ ]:
from data import Parser

parser = Parser()

text = 'In fact, 11% of American citizens, i.e. 21 million people of voting age, do not possess a photo ID card issued by a government agency of their State.'

spacy_tokens = parser.get_spacy_tokens(text)
print(spacy_tokens)

spacy_spans = parser.get_spans(text, spacy_tokens)
print(spacy_spans)

marian_tokens = parser.get_marian_tokens(text)
print(marian_tokens)

marian_spans = parser.get_spans(text, marian_tokens)
print(marian_spans)

alignment = parser.get_alignment(text, spacy_tokens, marian_tokens)

about 5% of our data is jank because of bad alignments!!!! (not really, fixed with percentage edge cases)

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')
doc = nlp(text)

In [ ]:
from collections import defaultdict
reverse = defaultdict(list)
for k, v in alignment.items(): reverse[v].append(k)

In [ ]:
for i, tok in enumerate(seq):
    spacy_par = doc[alignment[i]]
    print(f'{i}: {tok["text"]} -> {spacy_par.i}: {spacy_par.text}')
    print(f'derives from {spacy_par.head.i}: {spacy_par.head.text}')
    print(f'reverse: {reverse[spacy_par.head.i]}')
    print()